In [ ]:
import numpy as np
import pandas as pd
import pickle as pkl
import tensorflow.keras as keras
from tensorflow.keras.layers import Input
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Concatenate, Dense
from tensorflow.keras.optimizers import Adam


In [ ]:
from google.colab import drive
drive.mount('#')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
books_path = '#'
rate_path = '#'
user_path = '#'


books= pd.read_csv(books_path, error_bad_lines=False)
rate = pd.read_csv(rate_path, error_bad_lines=False)
user = pd.read_csv(user_path, error_bad_lines=False)

<ipython-input-5-39be3c5b91bf>:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books= pd.read_csv(books_path, error_bad_lines=False)
<ipython-input-5-39be3c5b91bf>:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books= pd.read_csv(books_path, error_bad_lines=False)
<ipython-input-5-39be3c5b91bf>:7: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  rate = pd.read_csv(rate_path, error_bad_lines=False)
<ipython-input-5-39be3c5b91bf>:8: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  user = pd.read_csv(user_path, error_bad_lines=False)


In [ ]:
books.head(5)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [ ]:
rate.head(5)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [ ]:
user.head(5)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [ ]:
print(books.shape)
print(rate.shape)
print(user.shape)

(271360, 8)
(1149780, 3)
(278858, 3)


In [ ]:
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [ ]:
user.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [ ]:
books.duplicated().sum()

0

In [ ]:
rate.duplicated().sum()

0

In [ ]:
user.duplicated().sum()

0

In [ ]:
ratings_with_name = rate.merge(books,on="ISBN")
ratings_with_name.shape

(1031136, 10)

In [ ]:
ratings_with_name.head(5)

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...


In [ ]:
# Preprocess data
n_users = rate['User-ID'].nunique()
n_books = rate['ISBN'].nunique()

user_ids = rate['User-ID'].astype("category").cat.codes.values
book_ids = rate['ISBN'].astype("category").cat.codes.values
ratings = rate['Book-Rating'].values

In [ ]:
X_train_user, X_test_user, X_train_book, X_test_book, y_train, y_test = train_test_split(
    user_ids, book_ids, ratings, test_size=0.2, random_state=42
)

In [ ]:
# Mendapatkan jumlah pengguna dan item
num_users = len(np.unique(np.concatenate([X_train_user, X_test_user])))
num_books = len(np.unique(np.concatenate([X_train_book, X_test_book])))


In [ ]:
# Membangun model
embedding_dim = 50

# Input layer untuk pengguna
user_input = Input(shape=(1,), name='user_input')
user_embedding = Embedding(input_dim=num_users, output_dim=embedding_dim)(user_input)
user_embedding = Flatten()(user_embedding)


In [ ]:
# Input layer untuk buku
book_input = Input(shape=(1,), name='book_input')
book_embedding = Embedding(input_dim=num_books, output_dim=embedding_dim)(book_input)
book_embedding = Flatten()(book_embedding)


In [ ]:
# Merangkai embedding pengguna dan buku
merged = Concatenate()([user_embedding, book_embedding])

In [ ]:
# Build model with smaller embedding size
embedding_size = 20  # Smaller embedding size

user_input = Input(shape=(1,), name='user_input')
user_embedding = Embedding(n_users, embedding_size, name='user_embedding')(user_input)
user_embedding = Flatten()(user_embedding)

book_input = Input(shape=(1,), name='book_input')
book_embedding = Embedding(n_books, embedding_size, name='book_embedding')(book_input)
book_embedding = Flatten()(book_embedding)

merged = Concatenate()([user_embedding, book_embedding])
output = Dense(1, activation='linear')(merged)

model = Model(inputs=[user_input, book_input], outputs=output)
model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))


In [ ]:
# Train model with smaller batch size
batch_size = 2048  # Anda dapat menyesuaikan ukuran batch sesuai kebutuhan

model.fit(
    x=[X_train_user, X_train_book],
    y=y_train,
    epochs=5,
    batch_size=batch_size,
    validation_data=([X_test_user, X_test_book], y_test)
)


Epoch 1/5
450/450 [==============================] - 72s 158ms/step - loss: 18.7458 - val_loss: 14.8831
Epoch 2/5
450/450 [==============================] - 66s 147ms/step - loss: 12.7401 - val_loss: 13.1537
Epoch 3/5
450/450 [==============================] - 64s 143ms/step - loss: 10.5242 - val_loss: 12.9285
Epoch 4/5
450/450 [==============================] - 63s 141ms/step - loss: 9.1975 - val_loss: 13.0908
Epoch 5/5
450/450 [==============================] - 63s 139ms/step - loss: 8.3477 - val_loss: 13.3003


In [ ]:
# Make predictions
predictions = model.predict([user_ids, book_ids])


35931/35931 [==============================] - 60s 2ms/step


In [ ]:
# Example: Recommend books for user_id 1
user_id_to_recommend = 2
user_rated_books = rate[rate['User-ID'] == user_id_to_recommend]['ISBN'].unique()

In [ ]:
# Filter out books that the user has already rated
unrated_books = rate[~rate['ISBN'].isin(user_rated_books)]['ISBN'].unique()

In [ ]:
# Create input for the model
user_input_for_recommendation = np.full(len(unrated_books), user_id_to_recommend)

# Transform ISBNs to book_ids using the same encoding as in the training data
book_ids_for_recommendation, _ = pd.factorize(unrated_books)

# Make predictions for unrated books
recommendation_scores = model.predict([user_input_for_recommendation, book_ids_for_recommendation])

10643/10643 [==============================] - 18s 2ms/step


In [ ]:
# Get top recommendations
top_recommendations_indices = recommendation_scores.flatten().argsort()[-5:][::-1]
top_recommendations_isbn = unrated_books[top_recommendations_indices]

In [ ]:
# Display recommended books
recommended_books = books[books['ISBN'].isin(top_recommendations_isbn)][['Book-Title', 'Image-URL-L']]

# Print the list of recommended books
print(f"\n Rekomendasi Buku Untuk User {user_id_to_recommend} berdasarkan Rating :\n")

for i, (title, url) in enumerate(recommended_books[['Book-Title', 'Image-URL-L']].values, start=1):
    print(f"{i}. Book Title: {title}")
    print(f"   Image URL: {url}\n")



 Rekomendasi Buku Untuk User 2 berdasarkan Rating :

1. Book Title: Futureland
   Image URL: http://images.amazon.com/images/P/0446610739.01.LZZZZZZZ.jpg

2. Book Title: Picknick der Friseure.
   Image URL: http://images.amazon.com/images/P/3499222841.01.LZZZZZZZ.jpg

3. Book Title: Divine Milieu
   Image URL: http://images.amazon.com/images/P/0060904879.01.LZZZZZZZ.jpg

4. Book Title: Arthur C. Clarke's Venus Prime 5
   Image URL: http://images.amazon.com/images/P/0743434722.01.LZZZZZZZ.jpg

